In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight')

In [2]:
data = pd.read_csv('market_day_level.csv',parse_dates=['trans_date'])
data = data.drop(['Unnamed: 0'],axis=1).rename(columns={'Unnamed: 0.1':'Weighted_Price'})

In [3]:
data

,trans_date,Weighted_Price,Open,High,Low,Close,Volume BTC,Volume USD
0,2017-01-01,719.5,977.253208,977.418333,977.095722,977.268465,6.313666,6215.613316
1,2017-01-02,2159.5,1012.263160,1012.541111,1011.959785,1012.273903,6.240900,6320.823541
2,2017-01-03,3599.5,1019.987431,1020.246257,1019.755146,1020.040472,6.866814,7006.929346
3,2017-01-04,5039.5,1076.552882,1077.291757,1075.523986,1076.553639,15.693348,17052.837734
4,2017-01-05,6479.5,1043.609160,1044.932715,1042.048319,1043.547951,25.459151,25625.674044
...,...,...,...,...,...,...,...,...
1535,2021-03-16,2209680.5,55342.874889,55388.028125,55297.895611,55343.697083,5.370296,295599.768827
1536,2021-03-17,2211120.5,56158.036813,56200.458931,56118.449000,56160.989236,4.055609,228489.573523
1537,2021-03-18,2212560.5,58475.809778,58511.486535,58437.665014,58476.000361,3.830878,224455.585754
1538,2021-03-19,2214000.5,58285.026007,58319.591563,58251.435347,58286.280590,2.446863,142674.673298


In [4]:
data['trans_date'] = data['trans_date'].dt.tz_localize(None)
data = data.groupby([pd.Grouper(key='trans_date', freq='D')]).first().reset_index()
data = data.set_index('trans_date')


In [5]:
data

,Weighted_Price,Open,High,Low,Close,Volume BTC,Volume USD
trans_date,,,,,,,
2017-01-01,719.5,977.253208,977.418333,977.095722,977.268465,6.313666,6215.613316
2017-01-02,2159.5,1012.263160,1012.541111,1011.959785,1012.273903,6.240900,6320.823541
2017-01-03,3599.5,1019.987431,1020.246257,1019.755146,1020.040472,6.866814,7006.929346
2017-01-04,5039.5,1076.552882,1077.291757,1075.523986,1076.553639,15.693348,17052.837734
2017-01-05,6479.5,1043.609160,1044.932715,1042.048319,1043.547951,25.459151,25625.674044
...,...,...,...,...,...,...,...
2021-03-16,2209680.5,55342.874889,55388.028125,55297.895611,55343.697083,5.370296,295599.768827
2021-03-17,2211120.5,56158.036813,56200.458931,56118.449000,56160.989236,4.055609,228489.573523
2021-03-18,2212560.5,58475.809778,58511.486535,58437.665014,58476.000361,3.830878,224455.585754


In [6]:
split_date = '2021-03-20'
data_train = data.loc[data.index <= split_date].copy()
data_test = data.loc[data.index > split_date].copy()

In [7]:
data_train

,Weighted_Price,Open,High,Low,Close,Volume BTC,Volume USD
trans_date,,,,,,,
2017-01-01,719.5,977.253208,977.418333,977.095722,977.268465,6.313666,6215.613316
2017-01-02,2159.5,1012.263160,1012.541111,1011.959785,1012.273903,6.240900,6320.823541
2017-01-03,3599.5,1019.987431,1020.246257,1019.755146,1020.040472,6.866814,7006.929346
2017-01-04,5039.5,1076.552882,1077.291757,1075.523986,1076.553639,15.693348,17052.837734
2017-01-05,6479.5,1043.609160,1044.932715,1042.048319,1043.547951,25.459151,25625.674044
...,...,...,...,...,...,...,...
2021-03-16,2209680.5,55342.874889,55388.028125,55297.895611,55343.697083,5.370296,295599.768827
2021-03-17,2211120.5,56158.036813,56200.458931,56118.449000,56160.989236,4.055609,228489.573523
2021-03-18,2212560.5,58475.809778,58511.486535,58437.665014,58476.000361,3.830878,224455.585754


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1540 entries, 2017-01-01 to 2021-03-20
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Weighted_Price  1540 non-null   float64
 1   Open            1540 non-null   float64
 2   High            1540 non-null   float64
 3   Low             1540 non-null   float64
 4   Close           1540 non-null   float64
 5   Volume BTC      1540 non-null   float64
 6   Volume USD      1540 non-null   float64
dtypes: float64(7)
memory usage: 96.2 KB


In [9]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))

def bit_pre_process(raw_data , seq_len, column = 1):
  
  #Select the feature/column 
  data = raw_data.iloc[:, column].values
  data = data.reshape(-1, 1)
  
  #Feature Scaling
  data = sc.fit_transform(data)
  
  #Making sequences
  
  X = []
  y = []

  for i in range(seq_len, len(data)):
      X.append(data[i-seq_len:i, 0])
      #print(X)
      y.append(data[i, 0])
  X, y = np.array(X), np.array(y)

  # Reshaping
  X = np.reshape(X, (X.shape[0], X.shape[1], 1))
  
  return X, y

In [10]:
#Setting the sequence length (Try different values)
sequence_length = 120

#Choosing the idex of the Close column
comumn_index= 4

#Preprocessing the training set
X_train, y_train = bit_pre_process(data_train , sequence_length, comumn_index)

In [11]:
X_train.shape

(1420, 120, 1)

In [12]:
y_train.shape

(1420,)

In [13]:
#Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#Initialising the RNN
regressor = Sequential()

#Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

#Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

#Adding the output layer
regressor.add(Dense(units = 1))

In [14]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse', 'mae', 'mape'])

In [15]:
regressor.fit(X_train, y_train, epochs = 50, batch_size = 50)

Epoch 1/50
29/29 [==============================] - 9s 153ms/step - loss: 0.0189 - mse: 0.0189 - mae: 0.0842 - mape: 69.7098
Epoch 2/50
29/29 [==============================] - 5s 156ms/step - loss: 0.0018 - mse: 0.0018 - mae: 0.0274 - mape: 22.0620
Epoch 3/50
29/29 [==============================] - 5s 167ms/step - loss: 0.0020 - mse: 0.0020 - mae: 0.0267 - mape: 20.0818
Epoch 4/50
29/29 [==============================] - 5s 168ms/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0230 - mape: 17.2014
Epoch 5/50
29/29 [==============================] - 5s 172ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0253 - mape: 19.5974
Epoch 6/50
29/29 [==============================] - 5s 169ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0234 - mape: 17.1493
Epoch 7/50
29/29 [==============================] - 5s 173ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0224 - mape: 15.8564
Epoch 8/50
29/29 [==============================] - 5s 175ms/step - loss: 0.0019 - mse: 0.0019 - mae: 0.0270 - mape: 19.6998


In [16]:
#Adding the previous 120 days cosing price to the test data
future_set = pd.concat((data_train.tail(sequence_length), data_test), axis = 0)

In [17]:
future_set

,Weighted_Price,Open,High,Low,Close,Volume BTC,Volume USD
trans_date,,,,,,,
2020-11-21,2045519.5,18711.404396,18720.903757,18701.215257,18712.128493,4.123268,77147.991595
2020-11-22,2046959.5,18431.999639,18443.227625,18420.851667,18431.881757,4.456401,81515.748425
2020-11-23,2048399.5,18448.265194,18460.097674,18435.513701,18448.214285,6.536172,120547.000194
2020-11-24,2049839.5,18862.930083,18876.027938,18848.626979,18862.424847,9.210299,174447.378583
2020-11-25,2051279.5,19055.788694,19068.754597,19042.230250,19056.251528,6.224043,118590.597544
...,...,...,...,...,...,...,...
2021-03-16,2209680.5,55342.874889,55388.028125,55297.895611,55343.697083,5.370296,295599.768827
2021-03-17,2211120.5,56158.036813,56200.458931,56118.449000,56160.989236,4.055609,228489.573523
2021-03-18,2212560.5,58475.809778,58511.486535,58437.665014,58476.000361,3.830878,224455.585754


In [18]:
#1st prediction based on the 120 values from training set
pred_list = []
data = future_set.iloc[:, 4].values
data = data.reshape(-1, 1)
#print(data)
#Feature Scaling
data = sc.fit_transform(data)
#a = 0
#b = 120

for i in range(0, 120):
  pred_list.append(data[i,0])

x_test = np.array(pred_list)

# Reshaping
x_test = np.reshape(x_test, (1,x_test.shape[0],1))
#x_test
predicted_stock_price = regressor.predict(x_test)
pred_list.append(predicted_stock_price[0,0]) #appending the predictions back into the list



    

In [19]:
#Future 60 days prediction - Each prediction will be appended into the list and used for the next prediction
a = 1 
b = 121
for i in range(0,60):
    #print(i)
    x_test = np.array(pred_list[a:b])
    x_test = np.reshape(x_test, (1,x_test.shape[0],1))
    #x_test
    predicted_stock_price = regressor.predict(x_test)
    pred_list.append(predicted_stock_price[0,0])
    a = a+1
    b=b+1
    
pred_list[120:180] #this is the prediction for next 60 days

[0.945139,
 0.9370278,
 0.9321582,
 0.93016016,
 0.93054134,
 0.9328847,
 0.93687916,
 0.94228023,
 0.9488556,
 0.9563511,
 0.9644777,
 0.97291833,
 0.981348,
 0.9894536,
 0.9969563,
 1.0036277,
 1.0093023,
 1.0138843,
 1.0173489,
 1.0197389,
 1.0211574,
 1.0217558,
 1.0217205,
 1.0212549,
 1.0205661,
 1.0198467,
 1.0192646,
 1.0189521,
 1.0190023,
 1.019465,
 1.0203503,
 1.021633,
 1.0232582,
 1.0251496,
 1.0272176,
 1.0293672,
 1.0315051,
 1.0335462,
 1.0354183,
 1.0370654,
 1.038451,
 1.0395569,
 1.040384,
 1.0409497,
 1.0412848,
 1.0414301,
 1.0414338,
 1.0413445,
 1.0412114,
 1.0410777,
 1.0409812,
 1.040949,
 1.0410008,
 1.0411454,
 1.0413829,
 1.0417057,
 1.0421,
 1.0425473,
 1.0430274,
 1.0435176]

In [20]:
predicted_stock_price = np.array(pred_list[120:180])
predicted_stock_price

array([0.945139  , 0.9370278 , 0.9321582 , 0.93016016, 0.93054134,
       0.9328847 , 0.93687916, 0.94228023, 0.9488556 , 0.9563511 ,
       0.9644777 , 0.97291833, 0.981348  , 0.9894536 , 0.9969563 ,
       1.0036277 , 1.0093023 , 1.0138843 , 1.0173489 , 1.0197389 ,
       1.0211574 , 1.0217558 , 1.0217205 , 1.0212549 , 1.0205661 ,
       1.0198467 , 1.0192646 , 1.0189521 , 1.0190023 , 1.019465  ,
       1.0203503 , 1.021633  , 1.0232582 , 1.0251496 , 1.0272176 ,
       1.0293672 , 1.0315051 , 1.0335462 , 1.0354183 , 1.0370654 ,
       1.038451  , 1.0395569 , 1.040384  , 1.0409497 , 1.0412848 ,
       1.0414301 , 1.0414338 , 1.0413445 , 1.0412114 , 1.0410777 ,
       1.0409812 , 1.040949  , 1.0410008 , 1.0411454 , 1.0413829 ,
       1.0417057 , 1.0421    , 1.0425473 , 1.0430274 , 1.0435176 ],
      dtype=float32)

In [21]:
#Inversing the scaled values 
predicted_stock_price = sc.inverse_transform(np.reshape(predicted_stock_price, (60,-1)))


In [22]:
predicted_stock_price

array([[58075.58 ],
       [57722.96 ],
       [57511.27 ],
       [57424.41 ],
       [57440.98 ],
       [57542.85 ],
       [57716.5  ],
       [57951.3  ],
       [58237.15 ],
       [58562.996],
       [58916.28 ],
       [59283.215],
       [59649.67 ],
       [60002.043],
       [60328.203],
       [60618.223],
       [60864.91 ],
       [61064.1  ],
       [61214.715],
       [61318.617],
       [61380.28 ],
       [61406.297],
       [61404.76 ],
       [61384.52 ],
       [61354.58 ],
       [61323.3  ],
       [61297.996],
       [61284.414],
       [61286.594],
       [61306.707],
       [61345.195],
       [61400.96 ],
       [61471.61 ],
       [61553.832],
       [61643.734],
       [61737.184],
       [61830.12 ],
       [61918.855],
       [62000.24 ],
       [62071.84 ],
       [62132.074],
       [62180.152],
       [62216.113],
       [62240.703],
       [62255.27 ],
       [62261.586],
       [62261.746],
       [62257.867],
       [62252.08 ],
       [62246.27 ],


In [23]:
pred_df = pd.DataFrame(predicted_stock_price)
pred_df

,0
0,58075.578125
1,57722.960938
2,57511.269531
3,57424.410156
4,57440.980469
5,57542.851562
6,57716.500000
7,57951.300781
8,58237.148438
9,58562.996094


In [ ]:
pred_df.to_csv('price_pred.csv', sep=',', encoding='utf-8')